# Getting Geographies for Analysis

Analysis frequently starts with identifying a geographic area for analysis. Many times this area is a standard geography such as a Core Based Statistica Area (CBSA) or a slightly larger Designated Market Area (DMA) around major metropolitan areas in the United States. Since the Business Analyst data is organized in countries, these can be discovered by interrogating the country of interest, and selecting the area of interest starts with selecting the country for analysis. Within this area of interest, many times smaller levels of geography are used for subsequent data aggregation and analysis, and by using a few available methods, smaller geographies can be selecting for subsequent analysis as well. Ultimately, getting block groups for an area of analysis such as Seattle should be as easy as programatically saying, "_I want block groups in Seattle._"

The first step is importing necessary modules. Long term, the idea is this will reside in the ArcGIS Python API, so instead of all the stuff in the cell below, getting started will be as easy as... 


```
from arcgis.dm import Country
from arcgis.dm.util import get_countries
```

Currently, while figuring out how to make all this work, it is not quite this easy, and there is an error message since I am adding some functionality onto the Spatially Enabled DataFrame, the `spatial` attribute for the Pandas DataFrame.

In [1]:
from pathlib import Path
import sys

project_parent = Path('./').absolute().parent
sys.path.append(str(project_parent/'src'))
from dm import Country
from dm.util import get_countries

D:\projects\demographic-modeling-module\src\dm\_modify_geoaccessor.py:31: UserWarning: registration of accessor <class 'dm._modify_geoaccessor.GeoAccessorIO'> under name 'spatial' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexistingattribute with the same name.
  class GeoAccessorIO(GeoAccessor):


## Get a Country

The first step is simply discovering what countries are available for analysis.

In [2]:
get_countries()

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2019,USA,2019


My machine has both data for the United States and Canada installed, so these are listed here. Using the three letter identifier from the `country` column, I can instantiate a Country object instance to work with.

In [3]:
usa = Country('USA')

usa

<class: Country - USA (local)>

## Get an Area of Interest

With the country instantiated, next we can select our area of analysis. For this example I am going to select the Seattle CBSA. First though, I want to see what geographic levels are available, and know how to reference the CBSAs.

In [4]:
usa.geographies

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
9,states,States,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...


The above geographies are listed heirarchially from smallest to largest area. While we later are going to retrieve the lowest geographic level, we first need to retrieve the Seattle CBSA as our area of interest. Before doing this, we can check to see what is available using a much faster method.

In [5]:
usa.cbsas.get_names('seattle')

0    Seattle-Tacoma-Bellevue, WA Metropolitan Stati...
Name: geo_name, dtype: object

Since this only returns one record, we can easily use this query string to get a Spatially Enabled DataFrame for the Seattle CBSA.

In [6]:
cbsa_df = usa.cbsas.get('seattle')

cbsa_df

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-13651055.7226, 5968866.240900002..."


Since a Spatially Enabled DataFrame, it is not difficult to display the results on a map to ensure it is the area we want to work with. In this case I take the extra step of setting the basemap since I like the dark gray.

NOTE: This will not display unless actively running in an environment with the web mapping widget configured.

In [7]:
wm01 = cbsa_df.spatial.plot()
wm01.basemap = 'dark-gray-vector'
wm01

MapView(layout=Layout(height='400px', width='100%'))

## Get Geographies for Analysis

The area of interest can now be used to retrieve the block groups for subsequent analysis.

In [8]:
bg_df = cbsa_df.block_groups.get()

bg_df.head()

,ID,NAME,SHAPE
0,530530714071,530530714.071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,"{""rings"": [[[-13654973.1668, 5957970.395400003..."


Frequently, we may simply want to access the lowest level of geography withough having to explicitly specify the name.

In [9]:
lvl_df = cbsa_df.level(0).get()

lvl_df.head()

,ID,NAME,SHAPE
0,530530714071,530530714.071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,"{""rings"": [[[-13654973.1668, 5957970.395400003..."


There also is the case when we need to select an area of interest, which is not a standard geography level just using a geometry. This can be accomplished usign a geometry object.

In this instance, I am just using the geometry from the CBSA, this is only as an example. Any polygon geometry can be used.

In [10]:
aoi_geom = cbsa_df.iloc[0]['SHAPE']

wthn_df = usa.level(0).within(aoi_geom)

wthn_df.head()

,ID,NAME,SHAPE
0,530530714071,530530714.071,"{""rings"": [[[-13618997.0451, 5953796.150899999..."
1,530530714072,530530714.072,"{""rings"": [[[-13621890.9066, 5953114.284599997..."
2,530530714073,530530714.073,"{""rings"": [[[-13622599.9004, 5953136.347999997..."
3,530530714112,530530714.112,"{""rings"": [[[-13627506.4183, 5953782.585000001..."
4,530530729061,530530729.061,"{""rings"": [[[-13654973.1668, 5957970.395400003..."


## Function Chaining for Efficiency

Once familiar with the workflow, the entire workflow can be chained for efficiency. This enables functionally being able to say, "_I want all block groups, or simply the lowest level of geography, in Seattle._"

In [11]:
sea_bg_df = usa.cbsas.get('seattle').level(0).get()

sea_bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2474 non-null   object  
 1   NAME    2474 non-null   object  
 2   SHAPE   2474 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.1+ KB


In this way, moving to a new area of interest is as simple as switching the selector to identify the name of the area to search for.

In [12]:
pdx_bg_df = usa.cbsas.get('portland').level(0).get()

pdx_bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1791 entries, 0 to 1790
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      1791 non-null   object  
 1   NAME    1791 non-null   object  
 2   SHAPE   1791 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 42.1+ KB


Using these methods makes it realatively straightforward to be able to quickly get relevant geographies for an area of interest - to be able to programatically say, "_I want the smallest geographies for a metro area,_" even if in a completely different country.

In [13]:
can = Country('CAN')

van_lvl_df = can.cmacas.get('vancouver').level(0).get()

van_lvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3442 entries, 0 to 3441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      3442 non-null   object  
 1   NAME    3442 non-null   object  
 2   SHAPE   3442 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 80.8+ KB


In [14]:
van_lvl_df.head()

,ID,NAME,SHAPE
0,59150369,59150369,"{""rings"": [[[-13700148.7313, 6317822.9978], [-..."
1,59150370,59150370,"{""rings"": [[[-13700774.3468, 6317681.446099997..."
2,59150373,59150373,"{""rings"": [[[-13701411.0943, 6317550.1294], [-..."
3,59150375,59150375,"{""rings"": [[[-13701423.339499999, 6317002.7137..."
4,59150376,59150376,"{""rings"": [[[-13701501.2631, 6316801.492200002..."


NOTE: Just as above, unless actively running in an environment with the web mapping widget configured, the mapping widget being created below will not display.

In [15]:
wm02 = van_lvl_df.spatial.plot()
wm02.basemap = 'dark-gray-vector'
wm02

MapView(layout=Layout(height='400px', width='100%'))

Finally, since a Spatially Enabled DataFrame, saving to a variety of formats including an Esri Feature Class is quite straightforward.

In [16]:
van_lvl_df.spatial.to_featureclass('../data/interim/interim.gdb/vancouver_lvl00')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\vancouver_lvl00'